In [ ]:
!pip install langchain faiss-cpu groq langchain-groq pdfplumber tiktoken


In [57]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA
from google.colab import files

import getpass
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter your OpenAI API key: ")
os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API key: ")


Enter your OpenAI API key: ··········
Enter your Groq API key: ··········


In [58]:
uploaded = files.upload()
pdf_path = list(uploaded.keys())[0]

loader = PyPDFLoader(pdf_path)
pages = loader.load()
splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(pages)

print(f"Loaded {len(pages)} pages")


Saving P Kaliraj (editor), Devi Thirupathi (editor) - Innovating with Augmented Reality_ Applications in Education and Industry-Auerbach Publications (2021).pdf to P Kaliraj (editor), Devi Thirupathi (editor) - Innovating with Augmented Reality_ Applications in Education and Industry-Auerbach Publications (2021) (4).pdf
Loaded 209 pages


In [ ]:
!pip install -U langchain-openai


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 70.4/70.4 kB 1.8 MB/s eta 0:00:00


In [59]:
# HuggingFace Embeddings
from langchain.embeddings import HuggingFaceEmbeddings
embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Creating FAISS vector store
from langchain.vectorstores import FAISS
vectordb = FAISS.from_documents(chunks, embedding)
retriever = vectordb.as_retriever()


In [60]:
# Connecting Groq LLM and create RetrievalQA chain
from langchain_groq import ChatGroq
from langchain.chains import RetrievalQA

llm = ChatGroq(model_name="llama3-70b-8192")

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever,
    return_source_documents=True
)


In [61]:
# Asking user input and get response from Groq
query = input("Ask a question about the document: ")

response = qa_chain(query)

print("\n Answer:")
print(response['result'])

print("\n Sources:")
for doc in response['source_documents']:
    print("-", doc.metadata)


Ask a question about the document: name ar devices

 Answer:
Based on the provided context, here are some AR devices mentioned:

1. Smartphones (e.g., Android phones)
2. Tablets
3. Head-mounted Displays (HMDs)
4. AR glasses (e.g., Vuzix Blade, Google, Apple, Facebook, Optinvent, Everysight Raptor, Epson Moverio)
5. Head-up Displays (HUD)
6. Magic Leap
7. AR contact lenses or Virtual Retinal Displays (VRD)
8. Handheld display (e.g., Renault Config car)
9. Stationary display (e.g., take an image, overlap objects, and display on TV)
10. Projected display (e.g., Sandbox)
11. Transparent smart window
12. Audio sets in museums

Please note that this is not an exhaustive list, and there may be other AR devices not mentioned in the provided context.

 Sources:
- {'producer': 'PyPDF', 'creator': 'Taylor and Francis', 'creationdate': '2021-10-28T14:00:09+00:00', 'author': 'Kaliraj P.', 'keywords': '', 'moddate': '2021-10-28T14:00:09+00:00', 'subject': 'Innovating with Augmented Reality', 'title'

In [62]:
def summarize_document(chunks, llm):
    from langchain.docstore.document import Document
    from langchain.chains import LLMChain
    from langchain.prompts import PromptTemplate

    docs = [Document(page_content=chunk.page_content) for chunk in chunks[:10]]
    prompt = PromptTemplate(
        input_variables=["text"],
        template="Summarize the following:\n\n{text}"
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    combined_text = " ".join([doc.page_content for doc in docs])
    return chain.run(combined_text).strip()
# Displaying the summary
summary = summarize_document(chunks, llm)
print(summary)


This is a book titled "Innovating with Augmented Reality: Applications in Education and Industry" edited by P. Kaliraj and T. Devi. The book was published in 2022 by CRC Press and explores the applications of Augmented Reality (AR) in education and industry.

The book consists of four chapters:

Chapter 1: "Augmented Reality" by M. Lissa and V. Bhuvaneswari provides an introduction to AR, its technology, hardware components, business applications, and tools available for AR development.

Chapter 2: "Industry 4.0: Augmented and Virtual Reality in Education" by P. Bhanu Prasad, N. Padmaj, B. Satish Kumar, and A. Aravind discusses the role of AR and Virtual Reality (VR) in Industry 4.0 and their applications in education.

Chapter 3: "Augmented Reality Changing the Food and Beverage Industries" by D. Ramya Chitra explores the applications of AR in the food and beverage industries, including its impact on these industries.

Chapter 4: "Augmented Reality: A Boon for the Teaching and Learnin